In this Colab, we will train an implementation of Dynaformer in PyG, mostly based on the Graphormer implementation at https://github.com/leffff/graphormer-pyg/blob/main/graphormer/layers.py, with some Dynaformer specific modifications.

In [2]:
%load_ext autoreload
%autoreload 2

The files with the model and test dataset will be imported from a GitHub repository by the following cell.

In [1]:
!git clone https://github.com/rbasto19/CS224W_final_project.git

Cloning into 'CS224W_final_project'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 32 (delta 15), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (32/32), 1.07 MiB | 1.24 MiB/s, done.
Resolving deltas: 100% (15/15), done.


Install necessary modules.

Import necessary modules.

In [64]:
import torch
from torch_geometric.datasets import MoleculeNet
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from torch.utils.data import Subset
from torch_geometric.utils.convert import to_networkx
import networkx as nx
from networkx import all_pairs_shortest_path
from sklearn.model_selection import train_test_split
import pickle
from torch_geometric.data import Data, Batch
import layers as layers
import model as model
from tqdm import tqdm
from torch_geometric.nn.pool import global_mean_pool


ModuleNotFoundError: No module named 'layers'

Load dataset (we start with a small dataset for testing purposes).

In [52]:
with open('/Users/rbasto/Stanford projects/CS224W/refined-set-2020-5-5-5_train_val.pkl', 'rb') as f:
  dataset = pickle.load(f)
from pypdb import get_all_info
from Bio import SeqIO
import requests
import time
from io import StringIO

def fetch_protein_sequence(pdb_id):
    """
    Fetch the protein sequence from the PDB database.
    """
    try:
        # Fetch FASTA sequence for the PDB ID
        url = f"https://www.rcsb.org/fasta/entry/{pdb_id}"
        response = requests.get(url)
        response.raise_for_status()
        # Parse the sequence from the response
        sequences = [record.seq for record in SeqIO.parse(StringIO(response.text), "fasta")]
        return str(sequences[0]) if sequences else None
    except Exception as e:
        print(f"Error fetching sequence for {pdb_id}: {e}")
        return None
    
sequences = {}
for i in range(len(dataset)):
  dataset[i] = Data(**dataset[i].__dict__)  # allowing to use different pyg version
  seq = fetch_protein_sequence(dataset[i].pdbid[:4])
  if seq not in sequences.keys():
    sequences[seq] = []
  sequences[seq].append(dataset[i].pdbid[:4])


Error fetching sequence for 3uri: HTTPSConnectionPool(host='www.rcsb.org', port=443): Max retries exceeded with url: /fasta/entry/3uri (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x3208fc790>: Failed to establish a new connection: [Errno 12] Cannot allocate memory'))


In [75]:
with open("/Users/rbasto/Downloads/md-refined2019-5-5-5/md-refined2019-5-5-5_test.pkl", 'rb') as f:
  dataset = pickle.load(f)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x10779baf0>>
Traceback (most recent call last):
  File "/Users/rbasto/miniconda3/envs/env224W/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [68]:
total_count = 0
for i in range(len(dataset)):
    for j in range(len(dataset[i])):
        dataset[i][j] = Data(**dataset[i][j].__dict__)  # allowing to use different pyg version
        dataset[i][j].x = dataset[i][j].x.to(torch.float32)

In [74]:
len(dataset[3])

100

In [47]:
with open('/Users/rbasto/Stanford projects/CS224W/sequences_data-6-6-6.pkl', 'rb') as f:
    sequences = pickle.load(f)

In [49]:
dataset = {}
for key in sequences.keys():
    for i in range(len(sequences[key])):
        sequences[key][i].x = sequences[key][i].x.to(torch.float32)

with open('/Users/rbasto/Stanford projects/CS224W/sequences_data-6-6-6.pkl', 'wb') as f:
    pickle.dump(sequences, f)

In [53]:
import torch
from copy import deepcopy
import torch_geometric.transforms as T

from pypdb import get_all_info
from Bio import SeqIO
import requests
import time
from io import StringIO

transform = T.Compose([T.remove_isolated_nodes.RemoveIsolatedNodes()])
def remove_random_edges(graph, p):    
    graph = deepcopy(graph)
    num_edges = int(graph.edge_index.size()[1] / 2)
    keep_edge = (torch.rand(num_edges) > p).reshape(-1,1)
    keep_edge = torch.hstack((keep_edge, keep_edge)).flatten()
    graph.edge_index = graph.edge_index.T[keep_edge].T
    graph.edge_attr = graph.edge_attr[keep_edge]
    graph = transform(graph)
    return graph

In [61]:
import random

In [63]:
a = [1,2,3,4,5,6]
b = random.sample(a, 3)
for i in range(len(b)):
    b[i] = 10000
print(a)
print(b)

[1, 2, 3, 4, 5, 6]
[10000, 10000, 10000]


In [60]:
for val in sequences.values():
    print(val)
    for i in range(len(val)):
        val[i] = remove_random_edges(val[i], 0.9)
    print(val)
    break

[Data(x=[0, 9], edge_index=[2, 0], edge_attr=[0, 3], y=[1], pos=[0, 3], pdbid='1y0x_pdbbind', num_node=[2], num_edge=[5], rfscore=[100], gbscore=[400], ecif=[1540], frame=-1, rmsd_lig=0.0, rmsd_pro=0.0), Data(x=[0, 9], edge_index=[2, 0], edge_attr=[0, 3], y=[1], pos=[0, 3], pdbid='1xzx_pdbbind', num_node=[2], num_edge=[5], rfscore=[100], gbscore=[400], ecif=[1540], frame=-1, rmsd_lig=0.0, rmsd_pro=0.0)]
[Data(x=[0, 9], edge_index=[2, 0], edge_attr=[0, 3], y=[1], pos=[0, 3], pdbid='1y0x_pdbbind', num_node=[2], num_edge=[5], rfscore=[100], gbscore=[400], ecif=[1540], frame=-1, rmsd_lig=0.0, rmsd_pro=0.0), Data(x=[0, 9], edge_index=[2, 0], edge_attr=[0, 3], y=[1], pos=[0, 3], pdbid='1xzx_pdbbind', num_node=[2], num_edge=[5], rfscore=[100], gbscore=[400], ecif=[1540], frame=-1, rmsd_lig=0.0, rmsd_pro=0.0)]


In [38]:
train_ids, val_ids = train_test_split([i for i in range(len(dataset))], test_size=0.3, random_state=42)
train_loader = DataLoader(Subset(dataset, train_ids), batch_size=32, shuffle=True)
val_loader = DataLoader(Subset(dataset, val_ids), batch_size=32, shuffle=True)

In [42]:
for data in train_loader:
    print(data.to_data_list()[0])
    break

AttributeError: 'GlobalStorage' object has no attribute 'to_data_list'

In [23]:
import random

In [31]:
random.sample(list(Subset(dataset, train_ids)), 2)

[[Data(x=[161, 9], edge_index=[2, 4880], edge_attr=[4880, 3], y=[1], pos=[161, 3], pdbid='4nw5_pdbbind', num_node=[2], num_edge=[5], rfscore=[100], gbscore=[400], ecif=[1540], frame=-1, rmsd_lig=0.0, rmsd_pro=0.0),
  Data(x=[119, 9], edge_index=[2, 3172], edge_attr=[3172, 3], y=[1], pos=[119, 3], pdbid='5d9l_pdbbind', num_node=[2], num_edge=[5], rfscore=[100], gbscore=[400], ecif=[1540], frame=-1, rmsd_lig=0.0, rmsd_pro=0.0),
  Data(x=[166, 9], edge_index=[2, 5032], edge_attr=[5032, 3], y=[1], pos=[166, 3], pdbid='4nw6_pdbbind', num_node=[2], num_edge=[5], rfscore=[100], gbscore=[400], ecif=[1540], frame=-1, rmsd_lig=0.0, rmsd_pro=0.0),
  Data(x=[136, 9], edge_index=[2, 3708], edge_attr=[3708, 3], y=[1], pos=[136, 3], pdbid='4nus_pdbbind', num_node=[2], num_edge=[5], rfscore=[100], gbscore=[400], ecif=[1540], frame=-1, rmsd_lig=0.0, rmsd_pro=0.0)],
 [Data(x=[84, 9], edge_index=[2, 1920], edge_attr=[1920, 3], y=[1], pos=[84, 3], pdbid='4dmn_pdbbind', num_node=[2], num_edge=[5], rfscore=

In [ ]:
for data in dataset:
    print(data)
    break

In [61]:
tasks = []
for val in sequences.values():
    if len(val) > 3:
        tasks.append(val)

In [45]:
import pypdb
found_pdbs = pypdb.Query('6cvv', query_type="structure").search()
print(found_pdbs[:10])

TypeError: 'NoneType' object is not subscriptable

Ligand-binding site visualization (visualization code generated with ChatGPT)

In [29]:
import plotly.graph_objects as go

data = dataset[0]
node_trace = go.Scatter3d(
    x=data.pos[:, 0],
    y=data.pos[:, 1],
    z=data.pos[:, 2],
    mode='markers',
    marker=dict(size=5, color='blue'),
    text=[f'Node {i}' for i in range(data.pos.size(0))]
)

# Lines for edges
edge_traces = []
for i, j in data.edge_index.t():
    edge_traces.append(go.Scatter3d(
        x=[data.pos[i, 0], data.pos[j, 0]],
        y=[data.pos[i, 1], data.pos[j, 1]],
        z=[data.pos[i, 2], data.pos[j, 2]],
        mode='lines',
        line=dict(color='black', width=2),
    ))

# Combine the traces and plot
fig = go.Figure(data=[node_trace] + edge_traces)
fig.update_layout(scene=dict(
    xaxis=dict(title='X'),
    yaxis=dict(title='Y'),
    zaxis=dict(title='Z'),
))
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Create the model and split dataset. Here we are using the default parameters in the Graphormer implementation for testing purposes.

In [5]:
graph_model = model.Graphormer(
    num_layers=2,
    input_node_dim=dataset[0].num_node_features,
    node_dim=32,
    input_edge_dim=dataset[0].num_edge_features,
    edge_dim=8,
    output_dim=1,
    n_heads=2,
    ff_dim=32,
    max_in_degree=4,
    max_out_degree=4,
    max_path_distance=4,
    num_heads_spatial=2
)

In [22]:
test_ids, train_ids = train_test_split([i for i in range(len(dataset))], test_size=0.5, random_state=42)
train_loader = DataLoader(Subset(dataset, train_ids), batch_size=4)
test_loader = DataLoader(Subset(dataset, test_ids), batch_size=4)

Define optimizer, loss function, and start training the model.

In [7]:
optimizer = torch.optim.AdamW(graph_model.parameters(), lr=1e-4)
loss_function = torch.nn.L1Loss(reduction="sum")

In [19]:
for epoch in range(10):
    graph_model.train()
    batch_loss = 0.0
    for batch in tqdm(train_loader):
        y = batch.y
        optimizer.zero_grad()
        output = global_mean_pool(graph_model(batch), batch.batch)
        loss = loss_function(output, y)
        batch_loss += loss.item()
        print(batch_loss)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(graph_model.parameters(), 1.0)
        optimizer.step()
    print("TRAIN_LOSS", batch_loss / len(train_ids))

    model.eval()
    batch_loss = 0.0
    for batch in tqdm(test_loader):
        y = batch.y

        with torch.no_grad():
            output = global_mean_pool(graph_model(batch), batch.batch)
            loss = loss_function(output, y)

        batch_loss += loss.item()

    print("EVAL LOSS", batch_loss / len(test_ids))

  2%|▏         | 3/156 [01:49<1:32:54, 36.43s/it]


KeyboardInterrupt: 

In [12]:
coords = torch.tensor([[-3.8230,  3.0390,  6.5360],
        [-3.4140,  3.3890,  5.3210],
        [-5.6860,  1.5970,  5.7010],
        [-4.6260,  1.7350,  6.5600],
        [-4.3070,  0.6970,  7.4450]])
norms = torch.tensor([[66.5701],
        [51.4538],
        [67.3824],
        [67.4437],
        [74.4641]])

In [23]:
for batch in train_loader:
    print(batch)
    break

DataBatch(x=[508, 9], edge_index=[2, 9260], edge_attr=[9260, 3], y=[4], pos=[508, 3], pdbid=[4], num_node=[8], num_edge=[20], rfscore=[400], gbscore=[1600], ecif=[6160], frame=[4], rmsd_lig=[4], rmsd_pro=[4], batch=[508], ptr=[5])


In [17]:
graph_model

Graphormer(
  (node_in_lin): Linear(in_features=9, out_features=32, bias=True)
  (edge_in_lin): Linear(in_features=3, out_features=8, bias=True)
  (centrality_encoding): CentralityEncoding()
  (spatial_encoding): SpatialEncoding()
  (layers): ModuleList(
    (0-1): 2 x GraphormerEncoderLayer(
      (attention): GraphormerMultiHeadAttention(
        (heads): ModuleList(
          (0-1): 2 x GraphormerAttentionHead(
            (edge_encoding): EdgeEncoding()
            (q): Linear(in_features=32, out_features=32, bias=True)
            (k): Linear(in_features=32, out_features=32, bias=True)
            (v): Linear(in_features=32, out_features=32, bias=True)
          )
        )
        (linear): Linear(in_features=64, out_features=32, bias=True)
      )
      (ln_1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (ln_2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (ff): Sequential(
        (0): Linear(in_features=32, out_features=32, bias=True)
        (1): 

In [15]:
norms - 2 * coords @ coords.T + norms.T

tensor([[   nan, 1.3289, 2.4995, 1.5316, 2.5584],
        [1.3289, 0.0092, 2.9185, 2.3958, 3.5434],
        [2.4995, 2.9185, 0.0000, 1.3713, 2.3986],
        [1.5316, 2.3958, 1.3713, 0.0000, 1.4009],
        [2.5584, 3.5434, 2.3986, 1.4009, 0.0055]])